In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as plot
import matplotlib as mpl
from matplotlib import gridspec
from matplotlib.dates import MonthLocator, DateFormatter
import seaborn as sns
import random
import datetime
from selenium.webdriver.chrome.options import Options

In [3]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
# from konlpy.tag import Okt
import re

In [5]:
from wordcloud import WordCloud
from collections import Counter

In [7]:
from transformers import AutoTokenizer

In [9]:
# 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 크롤링

### 하우스오브서바이벌
[삐맨](https://youtu.be/bXErVVhLTK0?si=7ufShyJCs0VJdh0q)
[고몽](https://youtu.be/ak78f8EOY14?si=SIWMbgZx14zReFd_)

- 검색어 : 하우스오브서바이벌
- 날짜, 댓글, 좋아요 수 3~4

### 삐맨

In [55]:
# 유튜브 영상 URL에 autoplay=0을 추가하여 자동 재생 방지
url = 'https://youtu.be/bXErVVhLTK0?si=7ufShyJCs0VJdh0q&autoplay=0'

driver = webdriver.Chrome()
driver.get(url)

In [57]:
# 영상을 일시정지(pause)
driver.execute_script("document.querySelector('video').pause();")

In [71]:
body = driver.find_element(By.CSS_SELECTOR,'body')

In [73]:
# 무한 스크롤링 2
while True:
    last_soup = bs(driver.page_source,'lxml')

    # 페이지 소스를 가져오는 거랑 오래 걸림
    # 한 번 END키 이후에 하면 오래 걸림
    # 따라서 그냥 END키 10번 하고 나서 비교하게 하기
    for i in range(10):
        body.send_keys(Keys.END)
        time.sleep(2)

    new_soup = bs(driver.page_source,'lxml') # 더 쉬운 방법 없나? 페이지 소스 불러오는 것도 오래 걸림

    if last_soup == new_soup :               # 비교 하는 것도 오래 걸림
        break
    else:
        pass

In [206]:
comments = driver.find_elements(By.ID,'comment')

In [208]:
len(comments)

717

In [257]:
id_list = []
date_list = []
comment_list = []
like_list = []

In [259]:
for comment in tqdm(comments):
    # 유저 이름
    user_id = comment.find_element(By.CSS_SELECTOR, '#author-text').text.strip()
    # 댓글 작성 날짜
    comment_date = comment.find_element(By.CSS_SELECTOR, '#published-time-text > a').text.strip()
    # 댓글 내용
    comment_text = comment.find_element(By.CSS_SELECTOR, '#content-text > span').text.strip()
    # 댓글 좋아요
    comment_like = comment.find_element(By.CSS_SELECTOR, '#vote-count-middle').text.strip()
    
    id_list.append(user_id)
    date_list.append(comment_date)
    comment_list.append(comment_text)
    like_list.append(comment_like)

In [261]:
len(id_list),len(date_list),len(comment_list),len(like_list)

(717, 717, 717, 717)

In [365]:
def cnt_str_to_int(cnt):
    if cnt == '':
        cnt = '0'
    if '천' in cnt:
        cnt = int(float(cnt.replace('천','')) * 1000)
    elif '만' in cnt:
        cnt = int(float(cnt.replace('만','')) * 10000)
    cnt = int(cnt)
    return cnt

In [367]:
data = {'id':id_list,'date':date_list,'comment':comment_list,'like':like_list}
df = pd.DataFrame(data)

In [371]:
df['like'] = df['like'].apply(cnt_str_to_int)

In [385]:
from datetime import datetime, timedelta

In [387]:
datetime.today()

datetime.datetime(2024, 10, 28, 11, 55, 17, 715199)

In [398]:
# 날짜 변환 함수
def convert_to_date(date_str):
    # 현재 날짜 기준으로 계산
    today = datetime.today().date()
    
    # "n개월 전" 패턴
    month_match = re.match(r'(\d+)개월 전', date_str)
    if month_match:
        months_ago = int(month_match.group(1))
        return today - timedelta(days=months_ago * 30)  # 월을 30일로 간주
    
    # "n주 전" 패턴
    week_match = re.match(r'(\d+)주 전', date_str)
    if week_match:
        weeks_ago = int(week_match.group(1))
        return today - timedelta(weeks=weeks_ago)
    
    # 변환 불가 시 NaT 반환
    return pd.NaT

In [400]:
# 날짜 형식 변환 적용
df['date_converted'] = df['date'].apply(convert_to_date)

In [423]:
df.sort_values('date')

,id,date,comment,like,date_converted
358,@lilacp4845,1개월 전,탈락자 3명 지정하는게 너무 리스크면 그냥 뽑기로 정하면 되잖아...,0,2024-09-28
328,@ngotrongZV,1개월 전,요즘 baccara 첫매 안주는곳 많은데 여기는 오퍼나준다는데 대박,0,2024-09-28
327,@user-ri2vp7qt7f,1개월 전,수시로 컷컷 하고 카메라와 조명을 다시 세팅하겠군요.,0,2024-09-28
326,@majjimajji574,1개월 전,마지막은 노엘을 투표하는게 남자들은 유리해보이는데 당연히 노엘을 투표해서 탈락시켰겠지,0,2024-09-28
325,@dspark2131,1개월 전,손세탁 시켰더니 한놈은 빨래판을 타블렛처럼 들고 있고 한놈은 통에 넣고 반바퀴씩 돌...,0,2024-09-28
...,...,...,...,...,...
680,@barkkyounghee,2개월 전(수정됨),하워드 저아저씨 우리쪽이야? 아님 저쪽이야? 바다쪽이야? 꼴보기 싫네 ㅋㅋㅋ\n외모...,0,2024-08-29
648,@이파리나무,2개월 전(수정됨),"저자들현재에살고있는사람들맞는지?\n세탁기가없다고,,아무리쇼라고는하지만시청자들얼땡이없...",0,2024-08-29
571,@blue714sky,3주 전,오징어게임을 이렇게까지..응용해서 하는구나...ㅎㅎㅎ,0,2024-10-07
170,@거울보고쓰러짐,4주 전,핸드폰 홍보를 이렇게 햇어야지,0,2024-09-30


In [419]:
df.like

0      1500
1      2400
2      1000
3      3200
4      2100
       ... 
712       0
713       0
714       0
715       0
716       0
Name: like, Length: 717, dtype: int64

In [408]:
df.comment

0                            PPL은 이렇게 하는거다라는게 느껴지네요 대단한듯
1                                 LG가 이런걸 하네..ㄷㄷ\n신박하네요.
2      탈락자 다시 돌아오는거 신박하다. 가족중에   집안일 1도 안하고 어지르기만하고 게...
3                       우리나라 기업들은 미국만 가면 마케팅 천재들이 되는거야..
4                        신박하네... 탈락하면 퇴장하는게 아니라 남아서 훼방놓네
                             ...                        
712    멍청하다고본다........가전은 그냥 어느회사나 만들수있는데... LG ?  물론...
713                                                    1
714    진부하다. 상금타려고 퉁수치고 팀먹고 배신하고....하다하다ㅡ이젠 탈락자 남겨서 끝...
715    근데 출연자들이 왜이리...못생겻냐.. 특히 여자들....남자들은 잘생긴편인데......
716                                       별 그지같은 예능이 다있네
Name: comment, Length: 717, dtype: object

In [416]:
# save
df.to_pickle("./data/youtube_bman.pickle")